# 经典召回模型

---
> （本学习笔记来源于DataWhale-第32期组队学习：[推荐系统](https://datawhalechina.github.io/fun-rec/#/ch02/ch2.1/ch2.1.1/usercf?id=%e5%9f%ba%e6%9c%ac%e6%80%9d%e6%83%b3)） ,
> [B站视频讲解](https://space.bilibili.com/431850986/channel/collectiondetail?sid=339597) 观看地址

**相关算法：**

- UserCF
- ItemCF
- Swing
- 矩阵分解

## 1.协同过滤算法

### 1.1 基本思想

协同过滤（Collaborative Filtering）推荐算法是最经典、最常用的推荐算法。基本思想是：

- 根据用户之前的喜好以及其他兴趣相近的用户的选择来给用户推荐物品。

  - 基于对用户历史行为数据的挖掘发现用户的喜好偏向，并预测用户可能喜好的产品进行推荐。
  - 一般是仅仅基于用户的行为数据（评价、购买、下载等）
- 目前应用比较广泛的协同过滤算法是基于领域的方法，主要有：

  - 基于用户的协同过滤算法（UserCF）: 给用户推荐和他兴趣相似的其他用户喜欢的产品。
  - 基于物品的协同过滤算法（ItemCF）: 给用户推荐和他之前喜欢的物品相似的物品。

不管是UserCF还是ItemCF算法，重点是计算用户之间（或物品之间）的相似度。

### 1.2 相似性度量方法

**1). 杰卡德（Jaccard）相似系数**

`Jaccard`系数是衡量两个集合的相似度的一种指标，计算公式如下：

![image.png](./assets/image.png)

- 其中$N(u),N(v)$分别表示用户$u$和用户$u$交互物品的集合, 若 $N(u),N(v)$为空，则$sim_{uv}$ = 1。
- 对于用户$u$和$u$，该公式反映了两个交互物品交集的数量占这两个用户交互物品并集的数量的比例。

用户杰卡德相似系数一般无法反映具体用户的评分喜好信息，所以常用来评估用户是否对某物品进行打分，而不是预估用户对某物品打多少分。

**Python实现相似度：** 

提示：`import jieba`出现找不到模块，在cmd中运行`pip install jieba`安装成功后再尝试。 

In [3]:
#import numpy as np
#from scipy.spatial.distance import pdist#直接调包可以计算JC值 :需要两个句子长度一样；所以暂时不用
import jieba  #如没有安装cmd中运行 pip install jieba 
 
def Jaccrad(model, reference):#terms_reference为源句子，terms_model为候选句子
    terms_reference= jieba.cut(reference)#默认精准模式
    terms_model= jieba.cut(model)
    grams_reference = set(terms_reference)#去重；如果不需要就改为list
    grams_model = set(terms_model)
    temp=0
    for i in grams_reference:
        if i in grams_model:
            temp=temp+1
    fenmu=len(grams_model)+len(grams_reference)-temp #并集
    jaccard_coefficient=float(temp/fenmu)#交集
    return jaccard_coefficient
 
a="香农在信息论中提出的信息熵定义为自信息的期望"
b="信息熵作为自信息的期望"
jaccard_coefficient=Jaccrad(a,b)
print(jaccard_coefficient)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\choi\AppData\Local\Temp\jieba.cache
Loading model cost 1.060 seconds.
Prefix dict has been built successfully.


0.38461538461538464


**2). 余弦相似度**

`余弦相似度`衡量了两个向量的夹角，夹角越小越相似。余弦相似度的计算如下，其与`杰卡德(Jaccard)`相似系数只是在分母上存在差异：

![image.png](./assets/1665557679443-image.png)

从向量的角度进行描述，令矩阵$A$为用户-物品交互矩阵，矩阵的行表示用户，列表示物品。
- 设用户和物品数量分别为$m,n$,交互矩阵$A$就是一个$m$行$n$列的矩阵。
- 矩阵中的元素均为0/1。若用户$i$对物品$j$存在交互，那么$A_{ij} = 1$,否则为0.
- 那么，用户之间的相似度可以表示为：
$$
   sim_{uv} = cos(u,v) = \frac{u.v}{|u|.|v|}
$$

向量$u,v$在形式都是one-hot类型，$u.v$表示向量点积。

上述用户-物品交互矩阵在现实中是十分稀疏的，为了节省内存，交互矩阵会采用字典进行存储。在`sklearn`中，余弦相似度的实现：

**在Python中使用sklearn计算余弦相似性**

`sklearn`提供内置函数`cosine_similarity()`可以直接用来计算余弦相似性。

In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
vec1 = np.array([1, 2, 3, 4])
vec2 = np.array([5, 6, 7, 8])

cos_sim = cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))
print(cos_sim[0][0])

0.9688639316269663


**在Python中使用numpy计算余弦相似性**

`numpy`模块没有直接提供计算余弦相似性的函数，我们可以根据余弦相似性的计算公式来计算。其中`numpy.doy()`函数可以计算两个向量的内积，`numpy.linalg.norm()`函数返回向量的模。

In [12]:
import numpy as np
vec1 = np.array([1, 2, 3, 4])
vec2 = np.array([5, 6, 7, 8])

cos_sim = vec1.dot(vec2) / np.linalg.norm(vec1) * np.linalg.norm(vec2)
print(cos_sim)

168.58232410309213


注意numpy只能计算numpy.ndarray类型向量的余弦相似性。

**3). 皮尔逊相关系数 **

在用户之间的余弦相似度计算时，将用户向量的内积展开为各元素乘积和：

$$ sim_{uv} = \frac{\sum_i r_{ui}*r_{vi}}{\sqrt{\sum_i r_{ui}^2}\sqrt{\sum_i r_{vi}^2}} $$



## 拓展学习
[协同过滤算法之通过Jaccard相似度计算推荐结果原理及代码实现](https://blog.csdn.net/sinat_31854967/article/details/118338888)
[基于用户的协同过滤来构建推荐系统](https://mp.weixin.qq.com/s/ZtnaQrVIpVOPJpqMdLWOcw)

